## Importación de los modulos y definición de variables

In [1]:
import csv
import requests
import pandas as pd
import io
from bs4 import BeautifulSoup
import os
import datetime
import numpy as np

In [2]:
pd.set_option('max_columns', None)

In [3]:
año = datetime.date.today().year
mes = datetime.date.today().month
dia = datetime.date.today().day

In [4]:
MESES = {1:'enero',
        2:'febrero',
        3:'marzo',
        4:'abril',
        5:'mayo',
        6:'junio',
        7:'julio',
        8:'agosto',
        9:'septiembre',
        10:'octubre',
        11:'noviembre',
        12:'diciembre'}

## Extracción y guardado de las tablas

In [5]:
urls = {
    'museos':'https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_4207def0-2ff7-41d5-9095-d42ae8207a5d',
    'salas_de_cine':'https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_392ce1a8-ef11-4776-b280-6f1c7fae16ae',
    'biblotecas_populares':'https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_01c6c048-dbeb-44e0-8efa-6944f73715d7'
    }

In [6]:
def descargar_csv_utf8(url):
    r = requests.get(url)
    s = BeautifulSoup(r.text, 'lxml')
    csv_url = s.find('a', attrs = {'class':'btn btn-green btn-block'}).get('href')
    r = requests.get(csv_url)
    r.encoding = 'utf-8'
    csvio = io.StringIO(r.text, newline="")
    data = []
    for row in csv.DictReader(csvio):
        data.append(row)
    df = pd.DataFrame(data)
    return df

In [7]:
def descargar_csv(url):
    r = requests.get(url)
    s = BeautifulSoup(r.text, 'lxml')
    csv_url = s.find('a', attrs = {'class':'btn btn-green btn-block'}).get('href')
    r = requests.get(csv_url)
    csvio = io.StringIO(r.text, newline="")
    data = []
    for row in csv.DictReader(csvio):
        data.append(row)
    df = pd.DataFrame(data)
    return df

In [8]:
dfs = {}
dfs['museos'] = descargar_csv(urls['museos'])

In [9]:
dfs['salas_de_cine'] = descargar_csv_utf8(urls['salas_de_cine'])
dfs['biblotecas_populares'] = descargar_csv_utf8(urls['biblotecas_populares'])

In [10]:
for key in dfs:
    año_mes_ruta = f'{key}/{año}-{MESES[mes]}'
    if not os.path.exists(key):
        os.mkdir(key)
        print("Directorio" , key ,  "creado")
        if not os.path.exists(año_mes_ruta):
            os.mkdir(año_mes_ruta)
    else:    
        print("Directorio" , key ,  "ya existe")

Directorio museos ya existe
Directorio salas_de_cine ya existe
Directorio biblotecas_populares ya existe


In [11]:
for key in dfs:
    dfs[key].to_csv(f'{key}/{año}-{MESES[mes]}/{key}-{dia}-{mes}-{año}.csv', index = False)

## Normalización de las tablas

### Tabla museos

In [12]:
dfs['museos']['id_departamento'] = np.nan
dfs['museos']['categoría'] = 'Museos'

In [13]:
df_museos = dfs['museos'][['localidad_id','provincia_id','id_departamento','categoría','provincia','localidad','nombre',
              'direccion','codigo_postal','telefono','mail','web','fuente']]

df_museos = df_museos.rename(columns = {'localidad_id':'cod_localidad',
                                        'provincia_id':'id_provincia',
                                        'direccion':'domicilio',
                                        'telefono':'número_de_telefono',
                                        'codigo_postal':'código_postal'}, inplace = False)

### Tabla salas_de_cine

In [14]:
df_salas_de_cine = dfs['salas_de_cine'][['Cod_Loc','IdProvincia','IdDepartamento','Categoría','Provincia','Localidad','Nombre',
              'Dirección','CP','Teléfono','Mail','Web','Fuente']]

df_salas_de_cine = df_salas_de_cine.rename(columns = {'Cod_Loc':'cod_localidad',
                                      'IdProvincia':'id_provincia',
                                      'IdDepartamento':'id_departamento',
                                      'Categoría':'categoría',
                                      'Provincia':'provincia',
                                      'Localidad':'localidad',
                                      'Nombre':'nombre',
                                      'Dirección':'domicilio',
                                      'CP':'código_postal',
                                      'Teléfono':'número_de_telefono',
                                      'Mail':'mail',
                                      'Web':'web',
                                      'Fuente':'fuente'}, inplace = False)

### Tabla biblotecas

In [15]:
df_biblotecas = dfs['biblotecas_populares'][['Cod_Loc','IdProvincia','IdDepartamento','Categoría','Provincia','Localidad','Nombre',
              'Domicilio','CP','Teléfono','Mail','Web','Fuente']]
df_biblotecas = df_biblotecas.rename(columns = {'Cod_Loc':'cod_localidad',
                                      'IdProvincia':'id_provincia',
                                      'IdDepartamento':'id_departamento',
                                      'Categoría':'categoría',
                                      'Provincia':'provincia',
                                      'Localidad':'localidad',
                                      'Nombre':'nombre',
                                      'Domicilio':'domicilio',
                                      'CP':'código_postal',
                                      'Teléfono':'número_de_telefono',
                                      'Mail':'mail',
                                      'Web':'web',
                                      'Fuente':'fuente'}, inplace = False)

### Tabla normalizada

In [16]:
df_normalizada = df_biblotecas.append(df_salas_de_cine).append(df_museos)
df_normalizada = df_normalizada.replace('s/d',np.nan)
df_normalizada = df_normalizada.replace('',np.nan)
df_normalizada['conteo'] = 1
df_normalizada.sample(10)

,cod_localidad,id_provincia,id_departamento,categoría,provincia,localidad,nombre,domicilio,código_postal,número_de_telefono,mail,web,fuente,conteo
1437,06427010,06,06427,Bibliotecas Populares,Buenos Aires,Virrey del Pino,Biblioteca Popular Madre Teresa,Comercio 6950,1763,443638,bibliotecamadreteresa@uolsinectis.com.ar,NaN,CONABIP,1
69,06161010,06,06161,Bibliotecas Populares,Buenos Aires,Carmen de Areco,Biblioteca Popular Juan Bautista Alberdi,Rivadavia 586,B6725BNP,440522,bibliocada@yahoo.com.ar,NaN,CONABIP,1
893,82126140,82,NaN,Museos,Santa Fe,San Jorge,Museo De La Ciudad,Bv. 25 de Mayo 810,S2452AYU,441557,NaN,NaN,DNPyM,1
1764,70028010,70,70028,Bibliotecas Populares,San Juan,Desamparados,Biblioteca Popular 1 de Febrero,Av.Paula A.de Sarmiento 498,5400,4231873,biblfeh2000@yahoo.com.ar,NaN,CONABIP,1
452,14098110,14,14098,Bibliotecas Populares,Córdoba,Elena,Biblioteca Popular Armando A.Telleria,Buteler 80,5815,4881566,bibatelleria@hotmail.com,NaN,CONABIP,1
4,02000010,02,02000,Bibliotecas Populares,Ciudad Autónoma de Buenos Aires,Ciudad de Buenos Aires,Biblioteca Popular Alberdi,Acevedo 666,C1414DJN,47753325,bibliotecapopularalberdi@yahoo.com.ar,NaN,CONABIP,1
290,06140010,06,06140,Bibliotecas Populares,Buenos Aires,Capitán Sarmiento,Biblioteca Popular Almafuerte,Av.Pte.Perón 467,2752,482271,bpalmafuerte@hotmail.com,NaN,CONABIP,1
638,46049070,46,NaN,Museos,La Rioja,Famatina,Museo Municipal De Famatina,Juan Facundo Quiroga S/N,5365,493340,NaN,NaN,DNPyM,1
1006,86147130,86,NaN,Museos,Santiago del Estero,Termas de Río Hondo,Museo Bellas Artes Y Artesanías,Juan B. Alberdi y Juan D. Perón,G4220ACS,NaN,NaN,NaN,DNPyM,1
301,02000010,02,NaN,Museos,Ciudad Autónoma de Buenos Aires,Ciudad Autónoma de Buenos Aires,Museo Etnográfico De Las Obras Misionales Pont...,Medrano 735,C1179AAM,48631357,ompar@sion.com,NaN,DNPyM,1


## Generación de las tablas requeridas

### Cantidad de registros totales por categoría

In [17]:
df_registros_totales = df_normalizada.groupby('categoría').sum(['conteo'])
df_registros_totales

,conteo
categoría,
Bibliotecas Populares,2017
Museos,1048
Salas de cine,329


### Cantidad de registros totales por fuente

In [18]:
df_provincia_categoría = df_normalizada.groupby(['provincia','categoría']).sum(['conteo'])
df_provincia_categoría

conteo
provincia                                          categoría                    
Buenos Aires                                       Bibliotecas Populares     543
                                                   Museos                    146
                                                   Salas de cine             106
Catamarca                                          Bibliotecas Populares      38
                                                   Museos                     30
...                                                                          ...
Tierra del Fuego                                   Salas de cine               3
Tierra del Fuego, Antártida e Islas del Atlánti... Museos                     14
Tucumán                                            Bibliotecas Populares      37
                                                   Museos                     25
                                                   Salas de cine               8

[73 rows x 1 columns]

### Cantidad de registros por provincia y categoría

In [19]:
df_fuente = df_normalizada.groupby(['fuente']).sum(['conteo'])
df_fuente

,conteo
fuente,
CONABIP,2010
DNPyM,1048
Gob. Pcia.,1
INCAA / SInCA,329
Secretaria de Cultura Provincial,6


### Cantidad de pantallas, butacas y espacios INCAA por provincia

In [20]:
df_cines_suma = dfs['salas_de_cine'].convert_dtypes()
df_cines_suma['Pantallas'] = df_cines_suma['Pantallas'].astype(int)
df_cines_suma['Butacas'] = df_cines_suma['Butacas'].astype(int)
df_cines_suma = df_cines_suma.rename(columns = {'Pantallas':'pantallas',
                                                'Butacas':'butacas'},inplace = False)

In [21]:
df_cines_suma1 = df_cines_suma.groupby(['Provincia']).sum(['pantallas','butacas'])
df_cines_suma2 = df_cines_suma[['Provincia','espacio_INCAA']].replace('',np.nan).groupby(['Provincia']).count()
df_cines_suma1['espacio_INCAA'] = df_cines_suma2
df_cines_suma1

,pantallas,butacas,espacio_INCAA
Provincia,,,
Buenos Aires,358,93112,20
Catamarca,12,3200,2
Chaco,14,2469,1
Chubut,10,2682,4
Ciudad Autónoma de Buenos Aires,153,31386,3
Corrientes,17,3370,1
Córdoba,105,20799,2
Entre Ríos,17,4086,2
Formosa,4,1184,1
